In [1]:
"""

啟用伺服器基本樣板

"""

# 引用Web Server套件
from flask import Flask, request, abort

# 從linebot 套件包裡引用 LineBotApi 與 WebhookHandler 類別
from linebot import (
    LineBotApi, WebhookHandler
)

# 引用無效簽章錯誤
from linebot.exceptions import (
    InvalidSignatureError
)

# 載入json處理套件
import json


# 設定Server啟用細節
app = Flask(__name__)

# 生成實體物件   #改自己的
line_bot_api = LineBotApi("")
handler = WebhookHandler("")

# 啟動server對外接口，使Line能丟消息進來
@app.route("/", methods=['POST'])
def callback():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']

    # get request body as text
    body = request.get_data(as_text=True)
    app.logger.info("Request body: " + body)

    # handle webhook body
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        abort(400)

    return 'OK'


In [2]:
"""
引用套件
連接資料庫

"""
import peewee


#改自己的
db = peewee.PostgresqlDatabase('ddmq64pcoa29to', 
                        user='mfpcpttczbibhf', 
                        password='bee6a9b9436c3a55e6ff290647370141cbbfd3b500b256186140f7a605b8420d',
                        host='ec2-52-202-146-43.compute-1.amazonaws.com', 
                        port=5432)

# 定義LineUserProfile 資料表
class LineUserProfile(peewee.Model):
    # 定義欄位
    displayName = peewee.CharField()
    pictureUrl = peewee.CharField()
    statusMessage = peewee.CharField()
    userId = peewee.CharField()
    
    # 指定使用的資料庫
    class Meta:
        database = db

# 創造資料庫
db.create_tables([LineUserProfile])


In [7]:
'''

撰寫用戶關注時，我們要處理的商業邏輯

1. 取得用戶個資，並存回伺服器
2. 把先前製作好的自定義菜單，與用戶做綁定
3. 回應用戶，歡迎用的文字消息與圖片消息

'''


# 載入Follow事件
from linebot.models.events import (
    FollowEvent
)

# 載入requests套件
import requests


# 告知handler，如果收到FollowEvent，則做下面的方法處理
@handler.add(FollowEvent)
def reply_text_and_get_user_profile(event):
    
    # 取出消息內User的資料
    user_profile = line_bot_api.get_profile(event.source.user_id)
    user_profile_dict = vars(user_profile)
    print(user_profile_dict,
        user_profile_dict.get("display_name"),
        user_profile_dict.get("picture_url"),
        user_profile_dict.get("status_message"),
        user_profile_dict.get("user_id")
    )
    new_user = LineUserProfile.create(
        displayName = user_profile_dict.get("display_name"),
        pictureUrl = user_profile_dict.get("picture_url") if user_profile_dict.get("picture_url") is not None else "",
        statusMessage = user_profile_dict.get("status_message") if user_profile_dict.get("status_message") is not None else "",
        userId = user_profile_dict.get("user_id")
    )
    

In [ ]:
'''

執行此句，啟動Server，觀察後，按左上方塊，停用Server

'''

if __name__ == "__main__":
    app.run(host='0.0.0.0')

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://0.0.0.0:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [08/Jun/2020 19:38:31] "POST / HTTP/1.1" 200 -


{'display_name': '李建德', 'user_id': 'Uf6a87bc14f8dacf9d85fdeb12227e9b5', 'picture_url': 'https://profile.line-scdn.net/0hCnXzgNoBHEFlAzSYVktjFllGEiwSLRoJHW1WcxNUR3RNMlIXDGdRcEQFQyMcO1oTDDBUdRMCF3MY', 'status_message': None} 李建德 https://profile.line-scdn.net/0hCnXzgNoBHEFlAzSYVktjFllGEiwSLRoJHW1WcxNUR3RNMlIXDGdRcEQFQyMcO1oTDDBUdRMCF3MY None Uf6a87bc14f8dacf9d85fdeb12227e9b5


127.0.0.1 - - [08/Jun/2020 19:38:36] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [08/Jun/2020 19:39:09] "POST / HTTP/1.1" 200 -


{'display_name': '昱融', 'user_id': 'Ua71c676ffbaa8ac2be0396aac213116e', 'picture_url': None, 'status_message': None} 昱融 None None Ua71c676ffbaa8ac2be0396aac213116e


...

127.0.0.1 - - [08/Jun/2020 19:39:15] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [08/Jun/2020 19:39:16] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [08/Jun/2020 19:39:17] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [08/Jun/2020 19:39:18] "POST / HTTP/1.1" 200 -


{'display_name': '朱祐震', 'user_id': 'U8c27681e5449cdf3796cf579d76cf132', 'picture_url': 'https://profile.line-scdn.net/0hz-M5Cy2aJQJwTAkOBhdaVUwJK28HYiNKCH49M1RPejJVezVTGS5rZQIcLzEPKWoHTi0-YgdOLmZe', 'status_message': 'It is always morning somewhere in the world.'} 朱祐震 https://profile.line-scdn.net/0hz-M5Cy2aJQJwTAkOBhdaVUwJK28HYiNKCH49M1RPejJVezVTGS5rZQIcLzEPKWoHTi0-YgdOLmZe It is always morning somewhere in the world. U8c27681e5449cdf3796cf579d76cf132


127.0.0.1 - - [08/Jun/2020 19:39:19] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [08/Jun/2020 19:39:19] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [08/Jun/2020 19:39:21] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [08/Jun/2020 19:39:21] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [08/Jun/2020 19:39:24] "POST / HTTP/1.1" 200 -


{'display_name': '薰惠🌸', 'user_id': 'U39e8e24bb07fa3fb77138e6f2662d4ce', 'picture_url': 'https://profile.line-scdn.net/0hICzY-xm5FmxHID5xSHJpO3tlGAEwDhAkP0NdCTJzHAxtFVFoKxEOAmRwHFtiFVM_ckZdWTUgQF0-', 'status_message': None} 薰惠🌸 https://profile.line-scdn.net/0hICzY-xm5FmxHID5xSHJpO3tlGAEwDhAkP0NdCTJzHAxtFVFoKxEOAmRwHFtiFVM_ckZdWTUgQF0- None U39e8e24bb07fa3fb77138e6f2662d4ce


127.0.0.1 - - [08/Jun/2020 19:39:25] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [08/Jun/2020 19:39:27] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [08/Jun/2020 19:39:28] "POST / HTTP/1.1" 200 -
